In [1]:
import ase
from ase.visualize import view
#import flatgraphene as fg
from ase.io import read,write 
from pyiron import ase_to_pyiron, pyiron_to_ase
import numpy as np 

In [395]:
# Read structures 
p_2_o_5 = read('/nfshome/deshmukh/vaibhav/project/vasp/convergence/k_conv/P2O5/mp-2452.cif', format='cif').repeat([1,1,1])
p_2_o_5.set_positions(np.random.rand(len(p_2_o_5), 3))
na_2_o = read('/nfshome/deshmukh/vaibhav/project/vasp/convergence/k_conv/Na2O/mp-2352.cif', format='cif').repeat([1,6,1])
na_2_o.set_positions(np.random.rand(len(na_2_o), 3))
si_o_2 = read('/nfshome/deshmukh/vaibhav/project/vasp/convergence/k_conv/SiO2/mp-546794.cif', format='cif').repeat([1,2,1])
si_o_2.set_positions(np.random.rand(len(si_o_2), 3))

In [385]:
print(p_2_o_5.get_chemical_formula(), na_2_o.get_chemical_formula(), si_o_2.get_chemical_formula())

O10P4 Na2O O8Si4


In [398]:
# Create empty cell of nasicon dimension 
from ase import Atoms, Atom
atoms_gr = Atoms(cell=[[5,0,0],[0,5,0], [0,0,5]])#new_cell)
atoms_gr += na_2_o #+ si_o_2 +p_2_o_5
atoms_gr.wrap(pbc=True)
#atoms_gr.get_scaled_positions()
atoms_gr += si_o_2
#atoms_gr.wrap(pbc=True)
#atoms_gr.get_scaled_positions()
atoms_gr += p_2_o_5
atoms_gr.wrap(pbc=True)
#atoms_gr.get_scaled_positions()
atoms_gr.center()
#atoms_gr.wrap(pbc=True)
#atoms_gr.get_scaled_positions()
print(atoms_gr.get_chemical_formula(empirical=True))
write('na3sipo6_amorphous.cif', atoms_gr.repeat([1,1,1]), format='cif')
atom = read('na3sipo6_amorphous.cif', format='cif')
print(atom.get_chemical_formula())    

Na3O6PSi
Na12O24P4Si4


In [399]:
from pyiron import Project, ase_to_pyiron
import matplotlib.pyplot as plt
import numpy as np
from pyiron import Project
from ase.io import read, write
from pyiron import ase_to_pyiron
import ase
import os
import time
import ase
from ase.visualize import view
from ase import Atoms, Atom
from jinja2 import Template
from ase.geometry import cell

In [400]:
pr = Project("/nfshome/deshmukh/pyiron/projects/NASICON/new_glass") 

In [401]:
# Customizing the potentials (New potentials) [Adjust as per the Requrnments]
import pandas
padone_potential = pandas.DataFrame({
  'Name': ['NASICON_Pedone'],
  'Filename': [[]],
  'Model': ['Custom'],
  'Species': [['O', 'Na', 'Zr', 'Si', 'P']],
  'Config': [['atom_style full\n',      # Given function contains morse parametrs, i.e. it has bonds 
              '## create groups ###\n',
              'group O type 1\n',
              'group Na type 2\n',
              'group Zr type 3\n',
              'group Si type 4\n',
              'group P type 5\n',
              '\n',
              '## set charges - beside manually ###\n',
              'set group O charge -1.2000\n',
              'set group Na charge 0.6000\n',
              'set group Zr charge 2.4000\n',
              'set group Si charge 2.4000\n',
              'set group P charge 3.0000\n',
              '\n',
              'pair_style hybrid/overlay morse 15.0 mie/cut 15.0 coul/long 15.0 beck 15.0\n',
              'pair_coeff * * coul/long\n',
              'pair_coeff 1 2 beck 5.0 0 0 0 0\n',
              'pair_coeff 1 3 beck 1.0 0 0 0 0\n',
              'pair_coeff 1 4 beck 1.0 0 0 0 0\n',
              'pair_coeff 1 5 beck 1.0 0 0 0 0\n',
              'pair_coeff 1 1 beck 22.0 0 0 0 0\n',
              'pair_coeff 1 2 mie/cut 5.0 1.0 12.0 0\n',
              'pair_coeff 1 3 mie/cut 1.0 1.0 12.0 0\n',
              'pair_coeff 1 4 mie/cut 1.0 1.0 12.0 0\n',
              'pair_coeff 1 5 mie/cut 1.0 1.0 12.0 0\n',
              'pair_coeff 1 1 mie/cut 22.0 1.0 12.0 0\n',
              'pair_coeff 1 2 morse 0.023363 1.763867 3.006315\n',
              'pair_coeff 1 3 morse 0.206237 2.479675 2.436997\n',
              'pair_coeff 1 4 morse 0.340554 2.006700 2.100000\n',
              'pair_coeff 1 5 morse 0.831326 2.585833 1.800790\n',
              'pair_coeff 1 1 morse 0.042395 1.379316 3.618701\n',
              'kspace_style ewald 1.0e-8\n']]
})   

In [402]:
lmp_input = """\
# This script will run the calculations in one run in pyiron.
# official work of Vaibhav Arun Deshmukh, TU-Darmstadt

#------------------------------Cook and Quench method---------------------------#

#---------------------------- Atomic setup -------------------------------------#

units metal
dimension 3
boundary p p p
atom_style full
read_data structure.inp
include potential.inp
neigh_modify     delay 0

#--------------------------Initialize velocity at high temperature------------#
velocity all create 600 87287

# Run a short equilibration
#fix 1 all nve
#timestep 1.0
#run 1000

#-------------------------------------------------------------------------------#
minimize        1.0e-8 1.0e-6 10000 100000

fix             1 all box/relax iso 0.0 vmax {{ step }} 
minimize        1.0e-8 1.0e-6 10000 100000
unfix           1
minimize        1.0e-8 1.0e-6 10000 100000

fix             1 all box/relax iso 0.0 vmax 0.001
minimize        1.0e-8 1.0e-6 10000 100000
unfix           1
minimize        1.0e-8 1.0e-6 10000 100000

fix             1 all box/relax iso 0.0 vmax 0.001
minimize        1.0e-8 1.0e-6 10000 100000
unfix           1
minimize        1.0e-8 1.0e-6 10000 100000

fix             1 all box/relax iso 0.0 vmax 0.001
minimize        1.0e-8 1.0e-6 10000 100000
unfix           1
minimize        1.0e-8 1.0e-6 10000 100000

fix             1 all box/relax iso 0.0 vmax 0.001
minimize        1.0e-8 1.0e-6 10000 100000
unfix           1
minimize        1.0e-8 1.0e-6 10000 100000

write_data      minimized.data
write_restart   restart.data

write_data   	data.lammps
#------------------------------------------------------------------------------#
"""

In [413]:
script_run_lmp = Template(lmp_input)

In [415]:
xx = read(filename='/nfshome/deshmukh/vaibhav/NaSICON_dft/database_glass/pbe_structure_5_hdf5/pbe_structure_5/CONTCAR', format='vasp')
xx.get_chemical_formula(empirical=True)
xx.repeat([7,7,7])
xx.center()
xx.wrap()
print(xx.get_chemical_formula(empirical=True))
write('na3si2po8_amorphopus_large.cif', xx, format='cif')
ll = read('na3si2po8_amorphopus_large.cif', format='cif')
print(ll.get_chemical_formula())

Na3O8PSi2
Na12O32P4Si8


In [419]:
bulk_struct = ase_to_pyiron(read(filename='na3si2po8_amorphopus_large.cif', format='cif')).repeat([3,3,2])
job_name = 'na3si2po8_amorphous_large_lmp'   
job_lmp = pr.create.job.Lammps(job_name, delete_existing_job=True)
job_lmp.structure = bulk_struct
job_lmp.potential = padone_potential
# add that it should write a restart file
job_lmp.input.control['write_data'] = 'restart.data'
job_lmp.input.control['write_restart'] = 'restart.data'
job_lmp.input.control.load_string(script_run_lmp.render(step=0.001))
#job_lmp.server.queue = "short_l2"    ### See the queues.yaml file for queue names. Standard queue on Lichtenberg is normal with a runtime limit of 24 hours.
#job_lmp.server.cores = 96             ### The number of cores you want for your job
job_lmp.run()

2024-06-20 16:43:24,277 - pyiron_log - WARNING - Unknown parameter (does not exist in block_dict): write_data


The job na3si2po8_amorphous_large_lmp was saved and received the ID: 32533


ParserError: Error tokenizing data. C error: Expected 6 fields in line 4, saw 7


In [26]:
pr.update_from_remote()

/nfshome/deshmukh/miniconda3/envs/vaibi/lib/python3.12/site-packages/pysqa/ext/remote.py:413: UserWarning: /home/vd80naku/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (

  warnings.warn(stderr.read().decode())
/nfshome/deshmukh/miniconda3/envs/vaibi/lib/python3.12/site-packages/pysqa/ext/remote.py:413: UserWarning: /home/vd80naku/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (

  warnings.warn(stderr.read().decode())
100%|██████████| 19/19 [00:00<00:00, 228.37it/s]
/nfshome/deshmukh/miniconda3/envs/vaibi/lib/python3.12/site-packages/pysqa/ext/remote.py:413: UserWarning: 
  warnings.warn(stderr.read().decode())
100%|██████████| 1/1 [00:00<00:00, 134.06it/s]
/nfshome/des